In [12]:
import os
import numpy as np
tmd = os.path.join("TMD", "IVDW_11", "0", "data_deepmd", "coord.raw")

In [3]:
"Te\n".rstrip(r"\n")

'Te\n'

In [2]:
from deepmd import DeepMD
import numpy as np

data = DeepMD("IVDW_11", "0")

In [3]:
print(type(data.features))
for name, data in data.features.__dict__.items():
    print(f"{name}: {data.shape}")

<class 'deepmd.DeepMDModel'>
box: (2000, 9)
coord: (2000, 54, 3)
energy: (2000, 1)
force: (2000, 54, 3)
type_map: (2, 1)
type: (54, 1)
virial: (2000, 9)
